**Optimal Replacement Policy (Belady Algorithm)** for Cache Management Policy

**Terminology**

**block_trace** - list of sequestial block request from CPU to memory/disk

**cold_miss** - compulsory misses when the caceh is empty and it is the first time a block has been requested

**cap_miss** - capacity miss when cache run out of space and had to **evict** a block from cache to bring the requested one into cache

**hit** - if block found in cache

**miss** - if block is not found in cache, also known as **fault**

We want to optimize (minimize) the number of faults (or maximize number of hits)

In [0]:
import numpy as np
import random
import pandas as pd
from tqdm import tqdm

In [0]:
#length of block trace
l = 20

#lowest value of block number 
b_low = 1

#highest value of block number
b_high = 7

block_trace = np.random.randint(b_low, b_high, size=(l)).tolist()

In [0]:
data = pd.read_csv('cheetah.cs.fiu.edu-110108-113008.1.blkparse', header=None)

In [0]:
data

,0
0,./PaxHeaders.15436/cheetah.cs.fiu.edu-110108-1...
1,28 ctime=1410563671.7201132
2,NaN
3,89966527950584 4253 nfsd 508516672 8 W 6 0 88b...
4,89966527996712 4253 nfsd 508516680 8 W 6 0 b5e...
5,89966528039628 4253 nfsd 508516688 8 W 6 0 57d...
6,89966740167027 4253 nfsd 519854376 8 W 6 0 442...
7,89966740221751 4253 nfsd 519854384 8 W 6 0 7a1...
8,89966740263716 4253 nfsd 519854392 8 W 6 0 a28...
9,89966740305212 4253 nfsd 519854400 8 W 6 0 4a5...


In [0]:
data = data[3:]

In [0]:
df = pd.DataFrame(data[0].str.split(' ',9).tolist())

In [0]:
blocktrace = df[3].tolist()
blocktrace = [int(x) for x in blocktrace]

In [0]:
blocktrace

[508516672,
 508516680,
 508516688,
 519854376,
 519854384,
 519854392,
 519854400,
 488846200,
 508455704,
 508455712,
 508455720,
 508500288,
 508500296,
 508500304,
 508516672,
 508516680,
 508516688,
 283193112,
 283193120,
 283193120,
 283193128,
 283193128,
 283193136,
 283193136,
 283193144,
 283193144,
 283193152,
 283193152,
 283193160,
 283193160,
 283193168,
 283193168,
 283193176,
 283193176,
 283193184,
 283193184,
 283193208,
 283193208,
 283193216,
 283193216,
 283193224,
 283193224,
 283193232,
 283193232,
 283193240,
 283193240,
 283193248,
 283193248,
 283193256,
 283193256,
 283193264,
 283193264,
 283193272,
 283193272,
 283193280,
 283193280,
 283193288,
 283193288,
 283193296,
 283193296,
 283193304,
 283193304,
 283193192,
 283193192,
 283193200,
 283193200,
 283193312,
 283193312,
 283193320,
 283193320,
 283193328,
 283193328,
 283193336,
 283193336,
 283193344,
 283193344,
 283193352,
 283193352,
 283193360,
 283193360,
 283193368,
 283193368,
 283193376,
 283

In [0]:
len(blocktrace)

1322897

In [0]:
len(set(blocktrace))

671732

In [0]:
#sample block trace we use for testing and consistency 
block_trace = [0,1,2,0,1,3,0,3,1,2,1]

In [0]:
def checkindex(item, ilist):
  
  try:
    return ilist.index(item)
  except ValueError:
    return -1 

In [0]:
def optimal(blocktrace, frame):
    cache = []

    hit, miss = 0, 0

    print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for i, block in enumerate(blocktrace):
        if block in cache:
            hit += 1
            pass

        elif len(cache) < frame:
            cache.append(block)
            miss += 1

        else:
            remain_block_trace = blocktrace[i:]

            indexed = [checkindex(j, remain_block_trace) for j in cache]

            indexed2 = list(filter(lambda a: a != -1, indexed))
      
            if len(indexed2) == 1:
                cache.pop()
                if remain_block_trace[indexed2[0]] not in cache:
                    cache.pop()
                    cache.append(remain_block_trace[indexed2[0]])
                    cache.append(block)
                    miss += 1

                elif len(indexed2) == 0:
                    cache.pop(0)
                    cache.append(block)
                    miss += 1

                else:
                    cache.remove(remain_block_trace[max(indexed2)])
                    cache.append(block)
                    miss += 1


        print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    hitrate = hit / (hit + miss)
    print("\n\nhitrate: ", hitrate)

    return hitrate


In [0]:
def FIFO(blocktrace, frame):
    '''
      FIFO - First In First Out
    '''
  
    cache = []

    hit, miss = 0, 0

    print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for block in blocktrace:
        if block in cache:
            hit += 1
            pass

        elif len(cache) < frame:
            cache.append(block)
            miss += 1

        else:
            cache.pop(0)
            cache.append(block)
            miss += 1

        print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    hitrate = hit / (hit + miss)
    print("\n\nhitrate: ", hitrate)
  
    return hitrate

In [0]:
block_trace

[3, 5, 5, 6, 1, 5, 3, 4, 1, 5, 5, 3, 5, 1, 1, 2, 1, 1, 1, 3]

In [0]:
def FIFO_np(blocktrace, frame):
    
    cache = np.zeros((frame), dtype=int)
    hit, miss = 0, 0
    
    #print("block", "\t\t hit", "\t\t miss", "\t\t cache", \
     #     "\t\tpointer")

    pointer = 0
    for block in tqdm(blocktrace):
        if block in cache:
            hit += 1
        
        elif frame - np.count_nonzero(cache) != 0:
            cache[np.where(cache==0)[0][0]] = block
            miss += 1
        
        else:
            cache[pointer % frame] = block
            pointer += 1
            miss += 1
        
      #  print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache,\
       #       "\t\t", pointer)
    
    hitrate = hit / (hit + miss)
    print(hitrate)
    
    return hitrate
            

In [0]:
block_trace

[5, 1, 1, 6, 1, 3, 5, 5, 6, 3, 6, 0, 3, 2, 3, 3, 6, 4, 1, 6]

In [0]:
FIFO(block_trace, 3)

block 		 hit 		 miss 		 cache
3 		 0 		 1 		 [3]
5 		 0 		 2 		 [3, 5]
5 		 1 		 2 		 [3, 5]
6 		 1 		 3 		 [3, 5, 6]
1 		 1 		 4 		 [5, 6, 1]
5 		 2 		 4 		 [5, 6, 1]
3 		 2 		 5 		 [6, 1, 3]
4 		 2 		 6 		 [1, 3, 4]
1 		 3 		 6 		 [1, 3, 4]
5 		 3 		 7 		 [3, 4, 5]
5 		 4 		 7 		 [3, 4, 5]
3 		 5 		 7 		 [3, 4, 5]
5 		 6 		 7 		 [3, 4, 5]
1 		 6 		 8 		 [4, 5, 1]
1 		 7 		 8 		 [4, 5, 1]
2 		 7 		 9 		 [5, 1, 2]
1 		 8 		 9 		 [5, 1, 2]
1 		 9 		 9 		 [5, 1, 2]
1 		 10 		 9 		 [5, 1, 2]
3 		 10 		 10 		 [1, 2, 3]


hitrate:  0.5


0.5

In [0]:
FIFO_np(block_trace, 3)

block 		 hit 		 miss 		 cache 		pointer
3 		 0 		 1 		 [3 0 0] 		 0
5 		 0 		 2 		 [3 5 0] 		 0
5 		 1 		 2 		 [3 5 0] 		 0
6 		 1 		 3 		 [3 5 6] 		 0
1 		 1 		 4 		 [1 5 6] 		 1
5 		 2 		 4 		 [1 5 6] 		 1
3 		 2 		 5 		 [1 3 6] 		 2
4 		 2 		 6 		 [1 3 4] 		 3
1 		 3 		 6 		 [1 3 4] 		 3
5 		 3 		 7 		 [5 3 4] 		 4
5 		 4 		 7 		 [5 3 4] 		 4
3 		 5 		 7 		 [5 3 4] 		 4
5 		 6 		 7 		 [5 3 4] 		 4
1 		 6 		 8 		 [5 1 4] 		 5
1 		 7 		 8 		 [5 1 4] 		 5
2 		 7 		 9 		 [5 1 2] 		 6
1 		 8 		 9 		 [5 1 2] 		 6
1 		 9 		 9 		 [5 1 2] 		 6
1 		 10 		 9 		 [5 1 2] 		 6
3 		 10 		 10 		 [3 1 2] 		 7
0.5


0.5

In [0]:
hitrateFIFO = FIFO_np(blocktrace, 35000)


  9%|▉         | 123296/1322897 [00:14<02:41, 7429.97it/s]


 21%|██        | 272416/1322897 [00:28<01:40, 10444.05it/s]


 32%|███▏      | 424176/1322897 [00:42<01:29, 10048.85it/s]


 44%|████▎     | 575679/1322897 [00:57<01:24, 8823.63it/s]


 56%|█████▌    | 738300/1322897 [01:12<01:07, 8717.98it/s]


 70%|██████▉   | 919460/1322897 [01:26<00:36, 11205.67it/s]


 81%|████████  | 1065262/1322897 [01:41<00:25, 9998.93it/s] 


 91%|█████████ | 1200193/1322897 [01:55<00:12, 9937.13it/s]


100%|██████████| 1322897/1322897 [02:07<00:00, 10353.24it/s]

0.1276645120519587


In [0]:
def random_replacement(blocktrace, frame):
  
    cache = []
    hit, miss = 0, 0

    print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for i, block in enumerate(blocktrace):
        if block in cache:
            hit += 1
            pass

        elif len(cache) < frame:
            cache.append(block)
            miss += 1

        else:
            random.shuffle(cache)
            cache.pop()
            cache.append(i)
            miss += 1


    print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)
   
    hitrate = hit / (hit + miss)
    print("\n\nhitrate: ", hitrate)
    
    return hitrate

In [0]:
random_replacement(block_trace, 3)

block 		 hit 		 miss 		 cache
0 		 0 		 1 		 [0]
1 		 0 		 2 		 [0, 1]
2 		 0 		 3 		 [0, 1, 2]
0 		 1 		 3 		 [0, 1, 2]
1 		 2 		 3 		 [0, 1, 2]
3 		 2 		 4 		 [1, 0, 5]
0 		 3 		 4 		 [1, 0, 5]
3 		 3 		 5 		 [5, 0, 7]
1 		 3 		 6 		 [5, 0, 8]
2 		 3 		 7 		 [0, 5, 9]
1 		 3 		 8 		 [9, 5, 10]


hitrate:  0.2727272727272727


In [0]:
def LFU(blocktrace, frame):
  
  cache = []
  hit, miss = 0, 0

  print("block", "\t\t hit", "\t\t miss", "\t\t cache")
  for i, block in enumerate(blocktrace):
    if block in cache:
      hit += 1
      pass

    elif len(cache) < frame:
      cache.append(block)
      miss += 1
      
    else:
      
      past_blocktrace = blocktrace[:i]
      cache_element_frequency = list(map(lambda x: past_blocktrace.count(x), cache))
      cache.pop(cache_element_frequency.index(min(cache_element_frequency)))
      cache.append(block)
      miss += 1
    
    print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)
   
  hitrate = hit / (hit + miss)
  print("\n\nhitrate: ", hitrate)

In [0]:
LFU(block_trace, 3)

block 		 hit 		 miss 		 cache
0 		 0 		 1 		 [0]
1 		 0 		 2 		 [0, 1]
2 		 0 		 3 		 [0, 1, 2]
0 		 1 		 3 		 [0, 1, 2]
1 		 2 		 3 		 [0, 1, 2]
3 		 2 		 4 		 [0, 1, 3]
0 		 3 		 4 		 [0, 1, 3]
3 		 4 		 4 		 [0, 1, 3]
1 		 5 		 4 		 [0, 1, 3]
2 		 5 		 5 		 [0, 1, 2]
1 		 6 		 5 		 [0, 1, 2]


hitrate:  0.5454545454545454


In [0]:
def LRU(blocktrace, frame):
  cache = []
  hit, miss = 0, 0

  print("block", "\t\t hit", "\t\t miss", "\t\t cache")
  for i, block in enumerate(blocktrace):
    if block in cache:
      hit += 1
      pass

    elif len(cache) < frame:
      cache.append(block)
      miss += 1
      
    else:
      past_blocktrace = blocktrace[:i]
      past_blocktrace.reverse()
      
      indexed = [checkindex(j, past_blocktrace) for j in cache]

      #print('index: ', indexed)
      cache.pop(indexed.index(max(indexed)))
      cache.append(block)
      miss += 1
    
    print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)
   
  hitrate = hit / (hit + miss)
  print("\n\nhitrate: ", hitrate)

In [0]:
LRU(block_trace, 3)

block 		 hit 		 miss 		 cache
0 		 0 		 1 		 [0]
1 		 0 		 2 		 [0, 1]
2 		 0 		 3 		 [0, 1, 2]
0 		 1 		 3 		 [0, 1, 2]
1 		 2 		 3 		 [0, 1, 2]
3 		 2 		 4 		 [0, 1, 3]
0 		 3 		 4 		 [0, 1, 3]
3 		 4 		 4 		 [0, 1, 3]
1 		 5 		 4 		 [0, 1, 3]
2 		 5 		 5 		 [1, 3, 2]
1 		 6 		 5 		 [1, 3, 2]


hitrate:  0.5454545454545454


In [0]:
l = np.zeros((14), dtype=int)

In [0]:
l[3] = 2

In [0]:
len(l) - np.count_nonzero(l)

13

l.shape

In [0]:
np.where(l == 0)[0][1]

1

In [0]:
np.where(l == 0)

(array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),)

In [0]:
l= np.array([1,2,3,0,6,7,0])

In [0]:
l

array([1, 2, 3, 0, 6, 7, 0])

In [0]:
np.where(l==0)[0].shape[0]

2

In [0]:
l2 = [1,2,3,4,5,6,9,9]
l2.index(3)

2

In [0]:
blocktrace.index(283193152)

26

In [0]:
if 3 in l:
    print('yes')

yes


In [0]:
l[1]

2